In [53]:
# Method 2
import numpy as np

def read_sparse_matrix_method2_crs(filename):
    with open(filename, "r") as f:
        n = int(f.readline().strip())  # Read dimension

        # Collect entries by row
        entries_by_row = [[] for _ in range(n)]

        for line in f:
            parts = line.strip().split(',')
            if len(parts) == 3:
                value, row, col = float(parts[0]), int(parts[1]), int(parts[2])
                entries_by_row[row].append((col, value))

        # Create CRS arrays
        values = []
        ind_col = []
        row_ptr = [0]

        for row_entries in entries_by_row:
            # Sort entries by column index
            row_entries.sort()

            # Combine entries with the same column index
            combined_entries = {}
            for col, val in row_entries:
                combined_entries[col] = combined_entries.get(col, 0) + val

            # Add entries to values and column indices
            for col, val in sorted(combined_entries.items()):
                values.append(val)
                ind_col.append(col)

            # Update row pointer
            row_ptr.append(len(values))

    return n, values, ind_col, row_ptr

# Example usage
n, values, ind_col, row_ptr = read_sparse_matrix_method2_crs("a_4.txt")

# Verify the size
print(f"Matrix size: {n}")

# Check first 10 values
print("First 10 values:", values[:10])

# Check first 10 column indices
print("First 10 column indices:", ind_col[:10])

# Check first 10 row pointers
print("First 10 row pointers:", row_ptr[:10])

def read_vector(filename):
    with open(filename, "r") as f:
        n = int(f.readline().strip())  # Read vector dimension
        b = np.array([float(f.readline().strip()) for _ in range(n)])  # Read values
    return b

# Example usage
b = read_vector("b_4.txt")

# Verify the size
print(f"Vector size: {len(b)} (should be {n})")

# Print first 10 elements to check correctness
print("First 10 elements of b:", b[:10])

def gauss_seidel_setup_crs(n):
    x = np.zeros(n)  # Initial guess: all zeros
    epsilon = 1e-10  # Precision threshold
    max_iterations = 10000  # Safety limit
    return x, epsilon, max_iterations

# Example usage
x, epsilon, max_iterations = gauss_seidel_setup_crs(n)

# Check initial values
print(f"Initial x vector size: {len(x)} (should be {n})")
print(f"Initial x first 10 values: {x[:10]} (should be all zeros)")
print(f"Epsilon (precision threshold): {epsilon}")
print(f"Max iterations: {max_iterations}")

def gauss_seidel_crs(n, values, ind_col, row_ptr, b, x, epsilon, max_iterations):
    """
    Gauss-Seidel method for solving Ax = b where A is in CRS format.
    Prints error at each iteration.
    """
    for iteration in range(max_iterations):
        x_old = x.copy()  # Store previous values for convergence check

        for i in range(n):
            row_start = row_ptr[i]
            row_end = row_ptr[i+1]

            sum_ax = 0.0
            diag_value = None

            for idx in range(row_start, row_end):
                col = ind_col[idx]
                value = values[idx]

                if col == i:
                    diag_value = value  # Diagonal element
                else:
                    sum_ax += value * x[col]  # Non-diagonal sum

            if diag_value is None or abs(diag_value) < 1e-10:
                raise ValueError(f"Zero diagonal element found at row {i}, cannot proceed with Gauss-Seidel.")

            x[i] = (b[i] - sum_ax) / diag_value  # Update x_i

        # Calculate change in solution
        diff = np.max(np.abs(x - x_old))

        # Calculate current residual
        residual = calculate_residual_crs(n, values, ind_col, row_ptr, x, b)

        # Print errors for this iteration
        print(f"Iteration {iteration+1}: change in x = {diff:.2e}, residual = {residual:.2e}")

        # Check for convergence
        if residual < epsilon:
            print(f"Converged in {iteration+1} iterations.")
            return x

    print(f"Reached max iterations ({max_iterations}).")
    return x


def calculate_residual_crs(n, values, ind_col, row_ptr, x, b):
    """Calculate the residual ||Ax - b||∞ for a matrix in CRS format."""
    Ax = np.zeros(n)
    for i in range(n):
        row_start = row_ptr[i]
        row_end = row_ptr[i+1]
        for idx in range(row_start, row_end):
            col = ind_col[idx]
            Ax[i] += values[idx] * x[col]

    return np.max(np.abs(Ax - b))


# Example usage
x_solution = gauss_seidel_crs(n, values, ind_col, row_ptr, b, x, epsilon, max_iterations)


# Check first 10 values of x after iterations
print("First 10 values of x:", x_solution[:10])

def verify_solution_crs(n, values, ind_col, row_ptr, x_solution, b):
    """Verify the solution by computing ||Ax - b||∞."""
    error = calculate_residual_crs(n, values, ind_col, row_ptr, x_solution, b)
    print("Verification ||Ax - b||∞:", error)
    return error


# Run verification
error = verify_solution_crs(n, values, ind_col, row_ptr, x_solution, b)


Matrix size: 80000
First 10 values: [40.5, 3.5, 0.75, 10.75, 0.75, 8.0, 6.75, 21.0, 11.0, 20.5]
First 10 column indices: [0, 5672, 6725, 8398, 26569, 28391, 28598, 1, 29162, 2]
First 10 row pointers: [0, 7, 9, 12, 22, 28, 31, 36, 49, 58]
Vector size: 80000 (should be 80000)
First 10 elements of b: [ 5124663.5   2239165.    2171216.75 10983351.75  6503832.75  2611808.5
  4506054.   11361932.5   6767226.5   2568078.75]
Initial x vector size: 80000 (should be 80000)
Initial x first 10 values: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] (should be all zeros)
Epsilon (precision threshold): 1e-10
Max iterations: 10000
Iteration 1: change in x = 1.43e+05, residual = 9.15e+06
Iteration 2: change in x = 9.42e+04, residual = 3.16e+06
Iteration 3: change in x = 4.00e+04, residual = 5.43e+05
Iteration 4: change in x = 1.18e+04, residual = 1.06e+05
Iteration 5: change in x = 3.31e+03, residual = 2.61e+04
Iteration 6: change in x = 5.09e+02, residual = 5.10e+03
Iteration 7: change in x = 1.27e+02, residual = 1.